* Ionic liquids— salts that melt below the boiling point of water.   
* Ionic liquids don't have a measurable vapor pressure at room temperature.     
* With typical vapor pressures in the range of 10^-10 pascal (10-14 psi), ionic liquids have essentially no vapor emissions and so look like excellent candidates for "green solvents" to replace hazardous, air-polluting organic solvents like acetone and benzene. 
* With dozens of anions and cations to choose from, they can be tailored to specific needs and may be particularly useful as solvents for biocatalysis.

In [2]:
import pandas as pd
import numpy as np
from salty import check_name
import salty
import pickle

density_all=pd.read_csv('density.csv') #all 30k data
d2=density_all
density_all=density_all.drop(['Molar volume, m<SUP>3</SUP>/mol','Specific volume, m<SUP>3</SUP>/kg'],axis=1)
salts=np.array(density_all.salt_name)
unique_salts=np.unique(salts)
len(unique_salts)
print('there are '+str(len(unique_salts)) + ' unique salts')

there are 744 unique salts


#### Database ~30,000 out of which 744 salts are unique, the rest are repeated at different Temperatures and Pressures

In [3]:
salts=[]
two=[]
three=[]
four=[]
more=[]
for i in unique_salts:
    A=i.split()              #sorting 2,3,4 and more ions into separate lists
    if len(A)==2:
        two.append(A)
    elif len(A)==3:
        three.append(A)
    elif len(A)==4:
        four.append(A)
    else:
        more.append(A)

print('There are '+ str(len(two))+' salts of 1 each') #looks fine and clean
print('There are '+ str(len(three))+' salts of 2 cations/1anion or 2 anions/1 cation each') #confirm which belongs to what
print('There are '+ str(len(four))+' salts of 2 each')
print('There are '+ str(len(more))+' salts of 2 or more each')

There are 668 salts of 1 each
There are 72 salts of 2 cations/1anion or 2 anions/1 cation each
There are 1 salts of 2 each
There are 3 salts of 2 or more each


### Using salty to generate smiles

In [4]:
cation2=[]
anion2=[]
error2_anion=[]
error2_cation=[]

for i in two:
    cation2.append(i[0])
    anion2.append(i[1])

for i in anion2:   #CHECKing CHECK_NAME FUNC FOR MISSING ANION OR CATION
    #print(i)
    try:
        check_name(i)
    except:
        UnboundLocalError
        error2_anion.append(i)
        #print(i)
for i in cation2:
    #print(i)
    try:
        check_name(i)
    except:
        UnboundLocalError
        error2_cation.append(i)

print('There are '+ str(len(set(error2_anion)))+' unique missing anions from the data base')
#print(error2_anion)
print('There are '+ str(len(set(error2_cation)))+ ' unique missing cations from the data base')
#len(set(error2_anion))

There are 48 unique missing anions from the data base
There are 141 unique missing cations from the data base


### Handling two word anions 

In [5]:
anion3=[]  #handling 2 word anions
cation3=[] 
count=0
for i in three:
    if 'sulfate' in i:
        anion3.append(i[1]+' '+i[2])
        cation3.append(i[0])
        count+=1
    elif 'phosphate' in i:
        anion3.append(i[1]+' '+i[2])
        cation3.append(i[0])
        count+=1
    elif 'phosphonate' in i:
        anion3.append(i[1]+' '+i[2])
        cation3.append(i[0])
        count+=1
    elif 'carbonate' in i:
        anion3.append(i[1]+' '+i[2])
        cation3.append(i[0])
        count+=1

print('handling '+ str(count)+ ' out of '+str(len(three)))

handling 60 out of 72


In [6]:
for n,i in enumerate(anion3): #fixing the space to make it compatible with database
    if i=='diethyl phosphate':
        anion3[n]='diethylphosphate'
    elif i=='dimethyl phosphate':
        anion3[n]='dimethylphosphate'


error3_anion=[]
for i in anion3:
        try:
            check_name(i)
        except:
            UnboundLocalError
            error3_anion.append(i)

error3_cation=[]
for i in cation3:
        try:
            check_name(i)
        except:
            UnboundLocalError
            error3_cation.append(i)



print('There are '+ str(len(set(error3_anion)))+ ' unique missing anions from the data base')
#error3_anion
print('There are '+ str(len(set(error3_cation)))+ ' unique missing cations from the data base')
#error3_cation
error3_anion

There are 2 unique missing anions from the data base
There are 19 unique missing cations from the data base


['dibutyl phosphate', 'sulfate tetrafluoroborate']

### Combining descriptors with the salts to 1 database

In [7]:
for i in d2.index:
    A=d2['salt_name'][i].split()
    if len(A)==2:
        pass
    else:
        d2.drop(i,axis=0,inplace=True)
d2=d2.drop(['Molar volume, m<SUP>3</SUP>/mol','Specific volume, m<SUP>3</SUP>/kg'],axis=1)

In [8]:
cationDescriptors = salty.load_data("cationDescriptors.csv")
cationDescriptors.columns = [str(col) + '-cation' for col in cationDescriptors.columns]
anionDescriptors = salty.load_data("anionDescriptors.csv")
anionDescriptors.columns = [str(col) + '-anion' for col in anionDescriptors.columns]

In [9]:
c=[]
a=[]
for i in d2.index:
    c1,a1=d2['salt_name'][i].split()
    c.append(c1)
    a.append(a1)
d2['name-cation']=c  
d2['name-anion']=a
d3=pd.merge(d2,cationDescriptors,on="name-cation", how="inner")
d4=pd.merge(d3,anionDescriptors,on='name-anion',how="inner")
print(d4.shape)
d4.head()

(26317, 196)


,"Pressure, kPa","Specific density, kg/m<SUP>3</SUP>","Temperature, K",salt_name,name-cation,name-anion,smiles-cation,steiger-cation,Marsili Partial Charges-cation,BalabanJ-cation,...,VSA_EState2-anion,VSA_EState3-anion,VSA_EState4-anion,VSA_EState5-anion,VSA_EState6-anion,VSA_EState7-anion,VSA_EState8-anion,VSA_EState9-anion,VSA_EState10-anion,Topliss fragments-anion
0,101.325,899.10,298.15,tetradecyl(trihexyl)phosphonium dicyanamide,tetradecyl(trihexyl)phosphonium,dicyanamide,C(CCCCCCCC[P+](CCCCCC)(CCCCCC)CCCCCC)CCCCC,777.0,777.0,4.337445,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,777.0
1,100.000,897.36,298.15,tetradecyl(trihexyl)phosphonium dicyanamide,tetradecyl(trihexyl)phosphonium,dicyanamide,C(CCCCCCCC[P+](CCCCCC)(CCCCCC)CCCCCC)CCCCC,777.0,777.0,4.337445,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,777.0
2,101.325,903.40,288.15,tetradecyl(trihexyl)phosphonium dicyanamide,tetradecyl(trihexyl)phosphonium,dicyanamide,C(CCCCCCCC[P+](CCCCCC)(CCCCCC)CCCCCC)CCCCC,777.0,777.0,4.337445,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,777.0
3,101.325,897.60,298.15,tetradecyl(trihexyl)phosphonium dicyanamide,tetradecyl(trihexyl)phosphonium,dicyanamide,C(CCCCCCCC[P+](CCCCCC)(CCCCCC)CCCCCC)CCCCC,777.0,777.0,4.337445,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,777.0
4,101.325,891.90,308.15,tetradecyl(trihexyl)phosphonium dicyanamide,tetradecyl(trihexyl)phosphonium,dicyanamide,C(CCCCCCCC[P+](CCCCCC)(CCCCCC)CCCCCC)CCCCC,777.0,777.0,4.337445,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,777.0


### Removing rows with NA values for density

In [10]:
nan_rows = d4[d4.isnull().T.any().T]
print('Removing '+str(nan_rows.shape[0]) +' rows')
d4=d4.dropna()
d4.shape
d4.to_pickle("salts+descriptors.pkl")
print('Final Shape: '+str(d4.shape))

Removing 326 rows
Final Shape: (25991, 196)
